In [ ]:
import os
import numpy as np
import pandas as pd
import absl.logging
from sklearn.metrics import accuracy_score
from nlp_embedding import GloVe
from nlp_classifier import NaiveBayes, SVM, XGBoost, CNN
from sklearn import preprocessing

absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
def train(data_x, data_y, nlp_embedding, nlp_classifier, label_encoder, epochs=1, batch_size=1000, model_dir='models', start_idx=0):
    print('Training...')
    data_y_enc = label_encoder.transform(data_y)
    classes = np.unique(data_y_enc)
    
    for epoch in range(epochs):
        print(f'Epoch: {str(epoch + 1)}/{str(epochs)}')
        for i in range(start_idx, data_x.shape[0], batch_size):
            
            if i + batch_size > data_x.shape[0]:
                j = data_x.shape[0]
            else:
                j = i + batch_size
            
            print(f'Processing rows: {i} - {j - 1}')

            embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
            nlp_classifier.partial_fit(embeddings, data_y_enc[i:j], classes=classes)
            nlp_classifier.save(os.path.join(model_dir, f'model_{nlp_embedding.name}_{nlp_classifier.name}'))
        start_idx = 0
    
    print('Success!')

In [ ]:
def test(data_x, nlp_embedding, nlp_classifier, label_encoder, batch_size=1000, predictions_dir='predictions', start_idx=0):
    print('Testing...')
    fname = os.path.join(predictions_dir, f'model_{nlp_embedding.name}_{nlp_classifier.name}.csv')
    predictions_all = []

    if start_idx == 0 and os.path.exists(fname):
        os.remove(fname)
    
    for i in range(start_idx, data_x.shape[0], batch_size):

        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        predictions_enc = nlp_classifier.predict(embeddings)
        predictions = label_encoder.inverse_transform(predictions_enc)
        
        predictions_all.extend(predictions)

        pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    
    print('Success!')    
    
    return predictions_all

In [ ]:
max_words = 400
dataset_name = 'metrolyrics'

In [ ]:
if not os.path.exists(f'{dataset_name}/models'):
    os.makedirs(f'{dataset_name}/models')

if not os.path.exists(f'{dataset_name}/predictions'):
    os.makedirs(f'{dataset_name}/predictions')

In [ ]:
train_data = pd.read_csv(f'data/train/{dataset_name}.csv')
test_data = pd.read_csv(f'data/test/{dataset_name}.csv')

In [ ]:
genres = np.unique(train_data.genre)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(genres)

In [ ]:
nlp_classifier = NaiveBayes()
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier, label_encoder, model_dir=f'{dataset_name}/models')
test(test_data.lyrics, nlp_embedding, nlp_classifier, label_encoder, predictions_dir=f'{dataset_name}/predictions')

In [ ]:
nlp_classifier = SVM()
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier, label_encoder, model_dir=f'{dataset_name}/models')
test(test_data.lyrics, nlp_embedding, nlp_classifier, label_encoder, predictions_dir=f'{dataset_name}/predictions')

In [ ]:
nlp_classifier = XGBoost(len(genres))
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier, label_encoder, model_dir=f'{dataset_name}/models')
test(test_data.lyrics, nlp_embedding, nlp_classifier, label_encoder, predictions_dir=f'{dataset_name}/predictions')

In [ ]:
nlp_classifier = CNN(max_words * 100, len(genres), 'adam')
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier, label_encoder, epochs=5, model_dir=f'{dataset_name}/models')
test(test_data.lyrics, nlp_embedding, nlp_classifier, label_encoder, predictions_dir=f'{dataset_name}/predictions')